In [24]:
USE_GPU = False
from typing import List, Tuple, Dict, Any, Optional
import seaborn as sns
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import statsmodels.api as sm
import re
import plotly.express as px
import tensorboard
import pandas as pd
import numpy as np
import seaborn as sns
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import matplotlib.pyplot as plt
import os
import glob
from typing import Dict
from statsmodels.tsa.filters.hp_filter import hpfilter
import re
from datetime import datetime
import plotly
import plotly.offline as pyo
import plotly.io as pio

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [25]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('mode.chained_assignment', None)


In [26]:
pio.renderers.default = 'notebook+pdf'
# pyo.init_notebook_mode()

In [27]:


TIME_STATS = [
    'sess_time/ended_time',
    'sess_time/session_minutes',
    'sess_time/time_cutoff',
    'sess_time/time_large',
    'sess_time/time_medium',
    'sess_time/time_small',
]

TIME_STATS_GRANULAR = [
    'ended_time', 
    'session_minutes', 
    'time_cutoff', 
    'time_large', 
    'time_medium', 
    'time_small'
]

SIZE_STATS_GRANULAR = [
    'ended_event',
    'session_size',
    'size_cutoff',
    'inc_small',
    'inc_medium',
    'inc_large'
]
    

In [110]:

def plot_vectors(df, x_name, y, title):

    fig = px.line(
        df,
        x=x_name,
        y=y,
        color='model',
        title=f'Training {title}',
    )
    
    fig.show()

def plot_vectors_multiple(df, y, title):
    fig = px.line(
        df,
        x='step',
        y=y,
        title=f'Training {title}',
    )
    
    # fig.show()
    return fig
    
def df_by_time_window(df, model_name):
    df = df.drop(columns=['Unnamed: 0'])
    df = df.rename(columns={'exp_runs': 'exp_count'})
    df = df.groupby(['exp_count']).mean().reset_index()
    
    df['model'] = model_name
    df['step'] = df['exp_count']
        
    print(df.columns)
    df['model'] = model_name
    for col in TIME_STATS_GRANULAR:
        cycle, trend = sm.tsa.filters.hpfilter(df[col], lamb=100)
        df[col] = trend 


    return df

def granular_stats(df, model, hp_filter_cols):
    df = df.groupby(['exp_runs']).mean().reset_index()
    df['time_stamp'] = pd.date_range(start='2023 06 07 00:00:00', end='2023 06 07 06:00:00', periods=len(df))
    df = df.set_index('time_stamp') \
        .resample('1T') \
        .mean() \
        .reset_index() \
        .drop(columns=['time_stamp'])
    df['model'] =  model
    df['step'] = df.index.values
    
    for col in hp_filter_cols:
        cycle, trend = sm.tsa.filters.hpfilter(df[col], lamb=100)
        df[col] = trend
        
    return df


def global_illegal_moves(df_matrix):
    illegal_move_matrix = {}
    for df_name, df_path in df_matrix.items():
        print(f'Calculating illegal move {df_name} -> {df_path}')
        df = pd.read_csv(df_path)
        df['illegal_move'] = df['ended_event'] == -1
        count_moves = df.groupby(['exp_runs']).size().reset_index(name='exp_run_count')
        illegal_moves = df.groupby(['exp_runs'])['illegal_move'].sum().reset_index(name='illegal_move_count')
        perc_illegal_moves = pd.merge(count_moves, illegal_moves, on='exp_runs')
        perc_illegal_moves['perc_illegal_moves'] = perc_illegal_moves['illegal_move_count'] / perc_illegal_moves['exp_run_count']
        illegal_move_matrix[df_name] = perc_illegal_moves
    
    max_episode = min([df['exp_runs'].max() for df in illegal_move_matrix.values()])
    
    for df_name, df in illegal_move_matrix.items():
        df = df[df['exp_runs'] <= 5_500]
        df['time_stamp'] = pd.date_range(start='2023 06 07 00:00:00', end='2023 06 07 06:00:00', periods=len(df))
        df = df.set_index('time_stamp') \
            .resample('1T') \
            .mean() \
            .reset_index() \
            .drop(columns=['time_stamp'])
        
        # cycle, trend = sm.tsa.filters.hpfilter(df['perc_illegal_moves'], lamb=100)
        # df['perc_illegal_moves'] = trend
        df['model'] = df_name
        illegal_move_matrix[df_name] = df
        
    
    out_df = pd.concat(illegal_move_matrix.values())
    out_df = out_df.rename(columns={'exp_runs': 'episode'})
    return out_df 

def global_perc_to_end(df_matrix, session_size_bound=None):
    move_to_end_matrix = {}
    for df_name, df_path in df_matrix.items():
        print(f'Calculating percentage done for {df_name} {df_path}')
        df = pd.read_csv(df_path)
        if session_size_bound:
            df = df[
                (df['session_size'] >= session_size_bound[0]) & (df['session_size'] <= session_size_bound[1])
            ]
        
        df['perc_to_end'] = df['ended_event'] / df['session_size']
        perc_to_end_summary = df.groupby(['exp_runs'])['perc_to_end'].mean().reset_index(name='perc_to_end')
        move_to_end_matrix[df_name] = perc_to_end_summary
        
    
    max_episode = min([df['exp_runs'].max() for df in move_to_end_matrix.values()])

    for df_name, df in move_to_end_matrix.items():
        df = df[df['exp_runs'] <= 5_500]
        df['time_stamp'] = pd.date_range(start='2023 06 07 00:00:00', end='2023 06 07 06:00:00', periods=len(df))
        df = df.set_index('time_stamp') \
            .resample('1T') \
            .mean() \
            .reset_index() \
            .drop(columns=['time_stamp'])
        
        # cycle, trend = sm.tsa.filters.hpfilter(df['perc_to_end'], lamb=100)
        # df['perc_to_end'] = trend
        df['model'] = df_name
        move_to_end_matrix[df_name] = df
        
    out_df = pd.concat(move_to_end_matrix.values())
    out_df = out_df.rename(columns={'exp_runs': 'episode'})
    
    return out_df


def global_perc_done(df_matrix, session_size_bound=None):
    done_matrix = {}
    for df_name, df_path in df_matrix.items():
        print(f'Calculating percentage finished for {df_name} from {df_path}')
        df = pd.read_csv(df_path)
        if session_size_bound:
            df = df[
                (df['session_size'] >= session_size_bound[0]) & (df['session_size'] <= session_size_bound[1])
            ]
        
        df['perc_done'] = df['ended_event'] == df['session_size']
        perc_done_summary = df.groupby(['exp_runs']).agg(
            done_count=('perc_done', 'sum'),
            sess_count=('perc_done', 'count')
        ) \
        .reset_index()
        perc_done_summary['perc_done'] = perc_done_summary['done_count'] / perc_done_summary['sess_count']
        done_matrix[df_name] = perc_done_summary
    
    # max_episode = min([df['exp_runs'].max() for df in done_matrix.values()])
    for df_name, df in done_matrix.items():
        df['perc_done_rolling'] = df.sort_values(by='exp_runs')['perc_done'] \
            .rolling(window=5, min_periods=1, center=True, closed='neither') \
            .mean() \
            .reset_index(drop=True)
        
        df['model'] = df_name
        done_matrix[df_name] = df
    
    out_df = pd.concat(done_matrix.values())
    out_df = out_df.rename(columns={'exp_runs': 'episode'})
    return out_df
    
    

    

PLOTLY_COLORS = plotly.colors.DEFAULT_PLOTLY_COLORS

def plot_granular(df, model_list, showlegend=False):
    plot_list = []
    for model, color in zip(model_list, PLOTLY_COLORS):
        plot_list.append(
            go.Scatter(
                x=df['episode'],
                y=df[model],
                name=model,
                line=dict(color=color),
                showlegend=showlegend
            )
        )
    
    
    return plot_list

                

In [91]:
TB_LOGS = 'dqn_tb'
CSV_LOGS = 'rl_results/dqn_csv'

In [92]:
csv_dirs = {
    'DQN LABEL CNN': 'dqn_csv/train/dqn_label_cnn.csv',
    'DQN PRED CNN': 'dqn_csv/train/dqn_pred_cnn.csv',
    'DQN PRED NO NORM': 'dqn_csv/train/dqn_pred_cnn_no_norm.csv',
    'DQN LABEL MLP': 'dqn_csv/train/dqn_label_mlp.csv',
    'DQN NONE CNN': 'dqn_csv/train/dqn_none_cnn.csv',
}



In [93]:
illegal_moves = global_illegal_moves(csv_dirs)

Calculating illegal move DQN LABEL CNN -> dqn_csv/train/dqn_label_cnn.csv
Calculating illegal move DQN PRED CNN -> dqn_csv/train/dqn_pred_cnn.csv
Calculating illegal move DQN PRED NO NORM -> dqn_csv/train/dqn_pred_cnn_no_norm.csv
Calculating illegal move DQN LABEL MLP -> dqn_csv/train/dqn_label_mlp.csv
Calculating illegal move DQN NONE CNN -> dqn_csv/train/dqn_none_cnn.csv


In [94]:
plot_vectors(illegal_moves, 'episode', 'perc_illegal_moves', 'Illegal Move Occurence')

In [95]:
perc_to_end = global_perc_to_end(csv_dirs)

Calculating percentage done for DQN LABEL CNN dqn_csv/train/dqn_label_cnn.csv
Calculating percentage done for DQN PRED CNN dqn_csv/train/dqn_pred_cnn.csv
Calculating percentage done for DQN PRED NO NORM dqn_csv/train/dqn_pred_cnn_no_norm.csv
Calculating percentage done for DQN LABEL MLP dqn_csv/train/dqn_label_mlp.csv
Calculating percentage done for DQN NONE CNN dqn_csv/train/dqn_none_cnn.csv


In [96]:
plot_vectors(perc_to_end, 'episode', 'perc_to_end', 'Proximity to End')

In [97]:
small_perc_end = global_perc_to_end(csv_dirs, session_size_bound=(1, 20))
mid_perc_end = global_perc_to_end(csv_dirs, session_size_bound=(20, 40))
large_perc_end = global_perc_to_end(csv_dirs, session_size_bound=(40, 60))
max_perc_end = global_perc_to_end(csv_dirs, session_size_bound=(60, 1_000_000))

Calculating percentage done for DQN LABEL CNN dqn_csv/train/dqn_label_cnn.csv
Calculating percentage done for DQN PRED CNN dqn_csv/train/dqn_pred_cnn.csv
Calculating percentage done for DQN PRED NO NORM dqn_csv/train/dqn_pred_cnn_no_norm.csv
Calculating percentage done for DQN LABEL MLP dqn_csv/train/dqn_label_mlp.csv
Calculating percentage done for DQN NONE CNN dqn_csv/train/dqn_none_cnn.csv
Calculating percentage done for DQN LABEL CNN dqn_csv/train/dqn_label_cnn.csv
Calculating percentage done for DQN PRED CNN dqn_csv/train/dqn_pred_cnn.csv
Calculating percentage done for DQN PRED NO NORM dqn_csv/train/dqn_pred_cnn_no_norm.csv
Calculating percentage done for DQN LABEL MLP dqn_csv/train/dqn_label_mlp.csv
Calculating percentage done for DQN NONE CNN dqn_csv/train/dqn_none_cnn.csv
Calculating percentage done for DQN LABEL CNN dqn_csv/train/dqn_label_cnn.csv
Calculating percentage done for DQN PRED CNN dqn_csv/train/dqn_pred_cnn.csv
Calculating percentage done for DQN PRED NO NORM dqn_c

In [98]:
small_perc_end_pivot = small_perc_end.pivot(index='episode', columns='model', values='perc_to_end').reset_index()
mid_perc_end_pivot = mid_perc_end.pivot(index='episode', columns='model', values='perc_to_end').reset_index()
large_perc_end_pivot = large_perc_end.pivot(index='episode', columns='model', values='perc_to_end').reset_index()
max_perc_end_pivot = max_perc_end.pivot(index='episode', columns='model', values='perc_to_end').reset_index()

large_perc_end_pivot = large_perc_end_pivot.dropna()
max_perc_end_pivot = max_perc_end_pivot.dropna()
large_perc_end_pivot = large_perc_end_pivot[large_perc_end_pivot['episode'] < 5000]

In [111]:
prox_end_fig = make_subplots(rows=2, cols=2, subplot_titles=('1-20', '20-40', '40-60', '60+'), print_grid=True)

locs = [(1, 1), (1, 2), (2, 1), (2, 2)]

small_plots = plot_granular(small_perc_end_pivot, csv_dirs.keys(), showlegend=True)
mid_plots = plot_granular(mid_perc_end_pivot, csv_dirs.keys())
large_plots = plot_granular(large_perc_end_pivot, csv_dirs.keys())
max_plots = plot_granular(max_perc_end_pivot, csv_dirs.keys())

for plot_list, loc in zip([small_plots, mid_plots, large_plots, max_plots], locs):
    for plot in plot_list:
        prox_end_fig.add_trace(plot, row=loc[0], col=loc[1])


prox_end_fig.update_layout(
    legend_title='Model',
    title_text='Proximity To Session End By Session Size',
    showlegend=True
)

prox_end_fig.update_xaxes(title_text='Episode', row=1, col=1)
prox_end_fig.update_xaxes(title_text='Episode', row=1, col=2)
prox_end_fig.update_xaxes(title_text='Episode', row=2, col=1)
prox_end_fig.update_xaxes(title_text='Episode', row=2, col=2)

prox_end_fig.update_yaxes(title_text='Perc Completed', row=1, col=1)
prox_end_fig.update_yaxes(title_text='Perc Completed', row=1, col=2)
prox_end_fig.update_yaxes(title_text='Perc Completed', row=2, col=1)
prox_end_fig.update_yaxes(title_text='Perc Completed', row=2, col=2)

prox_end_fig.show()

This is the format of your plot grid:
[ (1,1) x,y   ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [112]:
perc_finished = global_perc_done(csv_dirs)

Calculating percentage finished for DQN LABEL CNN from dqn_csv/train/dqn_label_cnn.csv
Calculating percentage finished for DQN PRED CNN from dqn_csv/train/dqn_pred_cnn.csv
Calculating percentage finished for DQN PRED NO NORM from dqn_csv/train/dqn_pred_cnn_no_norm.csv
Calculating percentage finished for DQN LABEL MLP from dqn_csv/train/dqn_label_mlp.csv
Calculating percentage finished for DQN NONE CNN from dqn_csv/train/dqn_none_cnn.csv


In [113]:
if 'perc_done' in perc_finished.columns:
    perc_finished = perc_finished.drop(columns=['perc_done'])

plot_vectors(perc_finished, 'episode', 'perc_done_rolling', 'Percentage of Episodes Completed')

In [114]:
small_perc_done = global_perc_done(csv_dirs, session_size_bound=(1, 20))
mid_perc_done = global_perc_done(csv_dirs, session_size_bound=(20, 40))
large_perc_done = global_perc_done(csv_dirs, session_size_bound=(40, 60))
max_perc_done = global_perc_done(csv_dirs, session_size_bound=(60, 1_000_000))

Calculating percentage finished for DQN LABEL CNN from dqn_csv/train/dqn_label_cnn.csv
Calculating percentage finished for DQN PRED CNN from dqn_csv/train/dqn_pred_cnn.csv
Calculating percentage finished for DQN PRED NO NORM from dqn_csv/train/dqn_pred_cnn_no_norm.csv
Calculating percentage finished for DQN LABEL MLP from dqn_csv/train/dqn_label_mlp.csv
Calculating percentage finished for DQN NONE CNN from dqn_csv/train/dqn_none_cnn.csv
Calculating percentage finished for DQN LABEL CNN from dqn_csv/train/dqn_label_cnn.csv
Calculating percentage finished for DQN PRED CNN from dqn_csv/train/dqn_pred_cnn.csv
Calculating percentage finished for DQN PRED NO NORM from dqn_csv/train/dqn_pred_cnn_no_norm.csv
Calculating percentage finished for DQN LABEL MLP from dqn_csv/train/dqn_label_mlp.csv
Calculating percentage finished for DQN NONE CNN from dqn_csv/train/dqn_none_cnn.csv
Calculating percentage finished for DQN LABEL CNN from dqn_csv/train/dqn_label_cnn.csv
Calculating percentage finished

In [115]:
small_perc_done_pivot = small_perc_done.pivot(index='episode', columns='model', values='perc_done_rolling').reset_index()
mid_perc_done_pivot = mid_perc_done.pivot(index='episode', columns='model', values='perc_done_rolling').reset_index()
large_perc_done_pivot = large_perc_done.pivot(index='episode', columns='model', values='perc_done_rolling').reset_index()
max_perc_done_pivot = max_perc_done.pivot(index='episode', columns='model', values='perc_done_rolling').reset_index()

large_perc_done_pivot = large_perc_done_pivot.dropna()
max_perc_done_pivot = max_perc_done_pivot.dropna()
large_perc_done_pivot = large_perc_done_pivot[large_perc_done_pivot['episode'] < 5000]

In [116]:
prox_end_fig = make_subplots(rows=2, cols=2, subplot_titles=('1-20', '20-40', '40-60', '60+'), print_grid=True)

locs = [(1, 1), (1, 2), (2, 1), (2, 2)]

small_perc_done = plot_granular(small_perc_done_pivot, csv_dirs.keys(), showlegend=True)
mid_perc_done = plot_granular(mid_perc_done_pivot, csv_dirs.keys())
large_perc_done = plot_granular(large_perc_done_pivot, csv_dirs.keys())
max_perc_done = plot_granular(max_perc_done_pivot, csv_dirs.keys())

for plot_list, loc in zip([small_perc_done, mid_perc_done, large_perc_done, max_perc_done], locs):
    for plot in plot_list:
        prox_end_fig.add_trace(plot, row=loc[0], col=loc[1])


prox_end_fig.update_layout(
    legend_title='Model',
    title_text='Percentage of Episode Completed by Episode and Episode Size',
    showlegend=True
)

prox_end_fig.update_xaxes(title_text='Episode', row=1, col=1)
prox_end_fig.update_xaxes(title_text='Episode', row=1, col=2)
prox_end_fig.update_xaxes(title_text='Episode', row=2, col=1)
prox_end_fig.update_xaxes(title_text='Episode', row=2, col=2)

prox_end_fig.update_yaxes(title_text='Perc Completed', row=1, col=1)
prox_end_fig.update_yaxes(title_text='Perc Completed', row=1, col=2)
prox_end_fig.update_yaxes(title_text='Perc Completed', row=2, col=1)
prox_end_fig.update_yaxes(title_text='Perc Completed', row=2, col=2)

prox_end_fig.show()

This is the format of your plot grid:
[ (1,1) x,y   ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]

